In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.colors as mcolor
import matplotlib as mpl
import plotly.express as px
import plotly.graph_objects as go
from Bio import Phylo
import scipy.stats as stats
import random
from   statsmodels.stats.multitest import multipletests
import statistics
import json

In [ ]:
####### parameters #######
mpl.rcParams['font.family']       = 'sans-serif'
mpl.rcParams['font.sans-serif']   = ["Helvetica","Arial","DejaVu Sans","Lucida Grande","Verdana"]
mpl.rcParams['figure.figsize']    = [4,3]
mpl.rcParams['font.size']         = 10
mpl.rcParams["axes.labelcolor"]   = "#000000"
mpl.rcParams["axes.linewidth"]    = 1.0 
mpl.rcParams["xtick.major.width"] = 1.0
mpl.rcParams["ytick.major.width"] = 1.0
cmap1 = plt.cm.tab10
cmap2 = plt.cm.Set3  
colors1 = [cmap1(i) for i in range(0,10)]
colors2 = [cmap2(i) for i in range(0,12)] 
##########################

In [ ]:
os.chdir("/Users/konnonaoki/Documents/backupped/Research/IwasakiLab/Data/MetabolicNetworkEvolution/experiment/NK_M0155")

dir_list = ["tables", "images", "itol"]
for directory in dir_list:
    try:
        os.mkdir(directory)
    except:
        None
os.getcwd()

In [ ]:
# Classess of KOs

proj_data_dir = "/Users/konnonaoki/Documents/backupped/Research/IwasakiLab/Data/MetabolicNetworkEvolution/experiment/NK_M0151"

df_path_ko = pd.read_table(proj_data_dir + "/tables/path_ko.txt", names = ['Pathway', 'KO'])
df_rn_ko = pd.read_table(proj_data_dir + "/tables/rn_ko.txt", names = ['Reaction','KO'])
df_md_ko = pd.read_table(proj_data_dir + "/tables/md_ko.txt", names = ['Module','KO'])
df_path_md = pd.read_table(proj_data_dir + "/tables/path_md.txt", names = ['Pathway','Module'])
ontology = json.load(open("/Users/konnonaoki/Documents/backupped/Research/IwasakiLab/Data/MetabolicNetworkEvolution/experiment/NK_M0151/json/ko00001.json"))

ontology_tree = Phylo.BaseTree.Tree(Phylo.BaseTree.Clade(name=ontology['name']))
root_clade    = Phylo.BaseTree.Clade(name=ontology['name'])
stack = [(ontology, root_clade)]

while len(stack) > 0:
    term, clade = stack.pop()
    if ('children' in term.keys()):
        for child in term['children']:
            child_clade = Phylo.BaseTree.Clade(name = child['name'])
            clade.clades.append(child_clade)
            stack.append((child, child_clade))

ontology_tree = Phylo.BaseTree.Tree(root_clade)

list_category_ko = []
for clade in ontology_tree.clade.clades[0].clades:
    for tip in clade.get_terminals():
        KO = tip.name.split()[0]
        if (KO[0] == 'K'):
            list_category_ko.append([clade.name, KO])
df_category_ko = pd.DataFrame(list_category_ko, columns = ['category', 'KO'])
st_category_ko = []
for clade in ontology_tree.clade.clades[0].clades:
    for tip in clade.get_terminals():
        KO = tip.name.split()[0]
        if (KO[0] == 'K'):
            list_category_ko.append([clade.name, KO])
df_category_ko = pd.DataFrame(list_category_ko, columns = ['category', 'KO'])
df_category_ko = df_category_ko[~df_category_ko.duplicated()]

df_ko_count = pd.DataFrame(df_category_ko.KO.value_counts())
set_ko_with_unique_category = set(df_ko_count[df_ko_count['KO']==1].index)
df_category_ko['unique'] = [(ko in set_ko_with_unique_category) for ko in df_category_ko.KO]
df_uniquecategory_ko = df_category_ko[df_category_ko['unique']]

# color of function categories

colors = ['#66C2A5', '#FC8D62', '#8DA0CB', '#E78AC3', '#555555', '#FC8D62', '#8DA0CB', '#E78AC3', '#66C2A5', '#FC8D62', '#000000']

cm_name = 'Set3' # B->G->R
cm = plt.get_cmap(cm_name)

df_category_ko_module = pd.merge(df_category_ko, df_md_ko, on = 'KO')
df_category_ko_module['Nko'] = 1
df_category_module_count = df_category_ko_module.groupby(['category', 'Module'], as_index = False).sum()
df_maxcategory_module = df_category_module_count.loc[df_category_module_count.groupby('Module')['Nko'].idxmax(),:].sort_values('category')
df_maxcategory_module = df_maxcategory_module.reset_index().loc[:, ['category', 'Module']]
df_category_color = pd.DataFrame([[category, i] for i, category in enumerate(df_maxcategory_module.category.unique())], columns = ["category", 'category_id'])
df_category_color['color'] = [mcolor.rgb2hex(cm(i)) for i in df_category_color['category_id']]
#df_category_color

df_category_ko_pathway = pd.merge(df_category_ko, df_path_ko, on = 'KO')
df_category_ko_pathway['Nko'] = 1
df_category_pathway_count = df_category_ko_pathway.groupby(['category', 'Pathway'], as_index = False).sum()
df_maxcategory_pathway = df_category_pathway_count.loc[df_category_pathway_count.groupby('Pathway')['Nko'].idxmax(),:].sort_values('category')
df_maxcategory_pathway = df_maxcategory_pathway.reset_index().loc[:, ['category', 'Pathway']]
df_category_rn = pd.merge(df_category_ko, df_rn_ko)[["category", "Reaction"]].drop_duplicates()
df_rn_Ntarget = pd.DataFrame(df_category_rn.Reaction.value_counts()).reset_index().rename(columns = {"index":"Reaction", "Reaction":"Ntarget"})
df_category_rn_Ntarget = pd.merge(df_category_rn, df_rn_Ntarget)
df_category_uniquern = df_category_rn_Ntarget[df_category_rn_Ntarget["Ntarget"] == 1]
df_category_uniquern

#### 種ごとの各モジュール内の反応数を辞書化

In [ ]:
# 種ごとの各モジュールの反応数を取得
sp2md2Nrn = {}
md_set = set()
sp_set  = set()
with open("dataset/gn_md_Nreaction.mlgtdb_MPPA.txt") as handle:
    for line in handle:
        words = line.split()
        sp       = words[0]; sp_set.add(sp)
        md     = words[1]; md_set.add(md)
        Nrn    = int(words[2])
        if sp not in sp2md2Nrn.keys():
            sp2md2Nrn[sp] = {}
        sp2md2Nrn[sp][md] = Nrn
def spmd2Nrn(sp, md, sp2md2Nrn):
    if(sp not in sp2md2Nrn.keys()):
        return 0
    elif (md not in sp2md2Nrn[sp].keys()):
        return 0
    else:
        return sp2md2Nrn[sp][md]

In [ ]:
df_Nreaction = pd.read_table("dataset/gn_md_Nreaction.mlgtdb_MPPA.txt", names = ["species", "module", "Nreaction"])
df_Nreaction

#### Prepare iTOL input files - Number of UNIQUE reactions in each functional category

In [ ]:
# Here, I only counted reactions which are unique to each functional category
df_sp_rn = pd.read_table("dataset/gn_rn.mlgtdb_MPPA.txt", names = ["species", "Reaction"])
df_sp_rn_category = pd.merge(df_sp_rn, df_category_uniquern)
df_sp_category_Nrn = df_sp_rn_category.groupby(["species","category"], as_index=False).count()
df_sp_category_Nrn

In [ ]:
for category in list(set(df_sp_category_Nrn["category"])):
    
    header = \
"DATASET_SYMBOL\n\
SEPARATOR SPACE\n\
DATASET_LABEL "+category.replace(" ","_")+"\n\
COLOR #ffff00\n\
DATA\n"
    
    df_sp_category_Nrn_ext = df_sp_category_Nrn[df_sp_category_Nrn["category"]==category]

    with open("itol/Nrn_" + category.replace(" ","_") + ".txt", "w") as handle:

        handle.write(header)

        for species, Nrn in zip(df_sp_category_Nrn_ext["species"], df_sp_category_Nrn_ext["Reaction"]):

            handle.write(species + " 2 " + str(Nrn) + " rgba(255,0,0,0.5) 1 1 " + category.replace(" ","_") + "\n")

#### 種ごとの各環境のhabitability scoreを辞書化

In [ ]:
# 種ごとの各環境のhabitability scoreを取得
# 97% identity, >150 bp coverage のデータを利用

df = pd.read_csv("/Users/konnonaoki/Documents/backupped/Research/IwasakiLab/Data/MetabolicNetworkEvolution/experiment/NK_M0135/prokatlas/processed/gn_MHI+header.97sim150ali.txt", sep= '\t', index_col=0)
sp2env2score = {}
list_sp_env_score = []
env_set = set()
for column in df.columns:
    for i, index in enumerate(df.index):
        sp         = index.split("_")[0].split(":")[1]; sp_set.add(sp)
        env       = column; env_set.add(env)
        score    = df[env][i]
        if not np.isnan(score):
            if sp not in sp2env2score.keys():
                sp2env2score[sp] = {}
            sp2env2score[sp][env] = score
            list_sp_env_score.append([sp, env, score])

df_sp_env_score = pd.DataFrame(list_sp_env_score, columns=['species', 'environment', 'score'])
df_sp_env_score

#### 環境habitat preference scoreと反応数

In [ ]:
result_list = []

for category in set(df_sp_category_Nrn["category"]):
    for env in list(set(df_sp_env_score["environment"])):
        df_sp_category_Nrn_ext = df_sp_category_Nrn[df_sp_category_Nrn["category"] == category].reset_index()[["species", "category", "Reaction"]]
        df_sp_env_score_ext = df_sp_env_score[df_sp_env_score["environment"] == env]

        df_sp_env_score_Nrn_ext = pd.merge(df_sp_env_score_ext, df_sp_category_Nrn_ext, on = "species", how = "left").fillna(0)
        corr_coeff = df_sp_env_score_Nrn_ext.score.corr(df_sp_env_score_Nrn_ext.Reaction)
        
        result_list.append([category, env, corr_coeff])
    
df_result_corr = pd.DataFrame(result_list, columns = ["category", "environment", "coefficient"])

In [ ]:
df_result_corr[df_result_corr["environment"] == "human_gut"]

In [ ]:
for category in set(df_sp_category_Nrn["category"]):

    df_result_corr_ext = df_result_corr[df_result_corr["category"] == category].sort_values("coefficient").reset_index()

    df_coeff = pd.DataFrame([])
    #df_coeff["environment"] = list(df_result_corr_ext["environment"][:5]) + list(df_result_corr_ext["environment"][-5:])
    #df_coeff["coefficient"] = list(df_result_corr_ext["coefficient"][:5]) + list(df_result_corr_ext["coefficient"][-5:])
    
    df_coeff["environment"] = list(reversed(list(df_result_corr_ext["environment"][-20:])))
    df_coeff["coefficient"] = list(reversed(list(df_result_corr_ext["coefficient"][-20:])))

    fig = plt.figure(figsize=(4,1.5))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    sns.barplot(x = 'environment', y = 'coefficient', data = df_coeff, color = "#F49991")
    ax.set_title(category)
    ax.set_xlabel("Environment")
    ax.set_ylim(0,0.55)
    plt.xticks(rotation=90)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    
    plt.savefig("images/NK_M0155_correlated_env_"+category+".pdf",bbox_inches='tight')
    plt.close()
    
    df_coeff = pd.DataFrame([])
    df_coeff["environment"] = list(reversed(list(df_result_corr_ext["environment"][:5]) + list(df_result_corr_ext["environment"][-5:])))
    df_coeff["coefficient"] = list(reversed(list(df_result_corr_ext["coefficient"][:5]) + list(df_result_corr_ext["coefficient"][-5:])))

    fig = plt.figure(figsize=(2,1.5))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    sns.barplot(x = 'environment', y = 'coefficient', data = df_coeff, palette = "coolwarm_r") # color = "#0096FF")
    ax.set_title(category)
    ax.set_xlabel("Environment")
    ax.set_ylim(-0.55,0.55)
    plt.xticks(rotation=90)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    
    plt.savefig("images/NK_M0155_correlated_env_"+category+"_top_bottom10.pdf",bbox_inches='tight')
    #plt.show()
    plt.close()

#### 獲得進化のパターンについて、特定の機能モジュールに注目

In [ ]:
# 各環境 vs map01220 (芳香族分解経路)で、habitability scoreと反応数の相関係数を算出

feature_list = [
    "M00419", "M00539",
    "M00537", "M00551",
    "M00538", "M00551",
    "M00418", "M00541",
    "M00543", "M00551",
    "M00547", "M00568",
    "M00548", "M00568",
    "M00551", "M00568",
    "M00637", "M00568",
    "M00547", "M00569",
    "M00548", "M00569",
    "M00551", "M00569",
    "M00544", "M00637",
    "M00637", "M00569",
    "M00534", "M00638",
    #"M00534", "M00569", # 本当は違うがこれまでも注目していたので
]

env_list = list(env_set)

#### モジュール内反応数と各環境のhabitatbility scoreとの相関をとる

In [ ]:
# Calculation of the correlation coefficient

list_module_env_coeff = []

for module in list(set(feature_list)):
    df_Nreaction_ext = df_Nreaction[df_Nreaction["module"]==module]
    
    for env in env_list:
        
        df_sp_env_score_ext = df_sp_env_score[df_sp_env_score["environment"]==env]

        df_sp_md_env_score_ext = pd.merge(df_Nreaction_ext, df_sp_env_score_ext, on = 'species', how = 'right')
        df_sp_md_env_score_ext['module'] = module
        df_sp_md_env_score_ext = df_sp_md_env_score_ext.fillna(0) # convert Nreaction = n.a. -> 0
        df_sp_md_env_score_ext

        if (np.var(df_sp_md_env_score_ext['Nreaction']) == 0 and np.var(df_sp_md_env_score_ext['score'])==0):
            coeff = 0
        else:
            coeff = df_sp_md_env_score_ext['Nreaction'].corr(df_sp_md_env_score_ext['score'])

        list_module_env_coeff.append([module, env, coeff])
        
df_module_env_coeff = pd.DataFrame(list_module_env_coeff, columns = ["Feature", "Environment", "Coefficient"])
df_module_env_coeff

In [ ]:
# Visualization of the result
for feature in feature_list:
    feature_env_coeff_dataframe_ext = df_module_env_coeff[df_module_env_coeff["Feature"]==feature]
    feature_env_coeff_dataframe_ext = feature_env_coeff_dataframe_ext.sort_values('Coefficient')

    fig = plt.figure(figsize=(15,2))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    sns.barplot(x = 'Environment', y = 'Coefficient', data = feature_env_coeff_dataframe_ext, palette="coolwarm")
    ax.set_title(feature)
    ax.set_xlabel("Environment")
    ax.set_ylim(-0.2,0.45)
    plt.xticks(rotation=90)
    plt.savefig("images/NK_M0155_func_habit_coeff_"+feature+".pdf",bbox_inches='tight')
    plt.close()

In [ ]:
# Analyze the result

env_list = list(set(df_module_env_coeff[df_module_env_coeff["Coefficient"] > 0.2]["Environment"]))

df_module_env_coeff[df_module_env_coeff["Coefficient"] > 0.2]

In [ ]:
# binarize habitability scores of extant species
df_bin = pd.DataFrame(index=df.index)
for env in env_list:
    for threshold in [0,5,10,15,20,25]:
        binary_list = []
        for score in df[env]:
            if np.isnan(score):
                binary_list.append("")
            elif (score > threshold):
                binary_list.append("1")
            else:
                binary_list.append("0")
        df_bin[env + str(threshold)] = binary_list

# indexをrenameして合わせる
old2new = {}
for old in df_bin.index:
    new = old.split("_")[0].split(":")[1]
    old2new[old]=new
df_bin=df_bin.rename(index=old2new)

#### FeatureとEnvironmentの相関をscatter plot

In [ ]:
# featureとenvの相関を、scatter plot
for feature in feature_list:
    df[feature] = [spmd2Nrn(idx.split(":")[1].split("_")[0], feature, sp2md2Nrn) for idx in df.index]
df.to_csv("tables/table2.csv")

md_list = list(set(feature_list))
env_list = ['soil', 'rhizosphere', 'biosolids', 'sludge','plant']

fig = plt.figure(figsize=(3,3))
for i, md in enumerate(md_list):
        for j, env in enumerate(env_list):
            ax = fig.add_axes([0.1+i*0.9,0.1+j*0.9,0.8,0.8])
            #ax = sns.violinplot( x=df["md:M00569"], y=df["soil"], linewidth=0)
            ax = sns.boxplot(x=df[md], y=df[env], color='white', linewidth=2)
            #ax = sns.swarmplot(x=df[md], y=df[env],alpha=1,size=1,color='black')
            ax.set_ylim(0,max(df[env]))
            if(i>0):
                ax.set_ylabel("")
                ax.tick_params(labelleft=False, left=False)
            if (j>0):
                ax.set_xlabel("")
                ax.tick_params(labelbottom=False, bottom=False)
plt.savefig("images/NK_M0155_Nrn_env_scactter.pdf", bbox_inches='tight')
plt.close()

In [ ]:
# rhizosphere と soilのhabitabilityの関係
fig = plt.figure(figsize=(3,3))
for i, md in enumerate(md_list):
        ax = fig.add_axes([0.1+i*0.9,0.1,0.8,0.8])
        sns.scatterplot(data=df, x="soil", y="rhizosphere", hue=md, ax=ax,alpha =0.1)
        if(i>0):
            ax.set_ylabel("")
            ax.tick_params(labelleft=False, left=False)
plt.savefig("images/fig3.pdf", bbox_inches='tight')
plt.close()

df['rhizosphere/soil'] = (df['rhizosphere'] / df['soil'])

fig = plt.figure(figsize=(3,3))
for i, md in enumerate(md_list):
        ax = fig.add_axes([0.1+i*0.9,0.1,0.8,0.8])
        sns.boxplot(data=df, x=md, y="rhizosphere/soil", ax=ax, color = 'white',linewidth=2)
        ax = sns.stripplot(x=df[md], y=df["rhizosphere/soil"],alpha=1,size=1,color='black', jitter=0.1)
        if(i>0):
            ax.set_ylabel("")
            ax.tick_params(labelleft=False, left=False)
plt.savefig("images/fig4.pdf", bbox_inches='tight')
plt.close()

In [ ]:
# habitability of ancestors + extant speciesを格納

sp2env2exist = {}
with open("/Users/konnonaoki/Documents/backupped/Research/IwasakiLab/Data/MetabolicNetworkEvolution/experiment/NK_M0135/pastml/env_gn.Bacteria.anc.txt") as handle:
    for line in handle:
        splitted = line.split("\t")
        env = splitted[0]
        sp   = splitted[1]
        exist = float(splitted[2])
        if sp not in sp2env2exist.keys():
            sp2env2exist[sp] = {}
        sp2env2exist[sp][env] = exist
def spenv2exist(sp, env, sp2env2exist):
    if sp not in sp2env2exist.keys():
        return 0
    elif env not in sp2env2exist[sp].keys():
        return 0
    else:
        return sp2env2exist[sp][env]

#### 環境進出との順序関係の検定

In [ ]:
# 系統樹読み込み
tree = Phylo.read("/Users/konnonaoki/Documents/backupped/Research/IwasakiLab/Data/MetabolicNetworkEvolution/experiment/NK_M0150/tree/bac120_msa_r89.faa.mlgtdb.representative.renamed.rooted.nwk",'newick')

#### 環境進出　→　反応獲得　の傾向があるか検定

In [ ]:
# 環境進出　→　反応獲得　の傾向があるか検定
# 直近の祖先がその環境に生息している枝で、反応が獲得される傾向があるか否か検定
# 環境生息の祖先状態が1の種を生息しているとし、環境生息の祖先状態が0の種を生息していないとする、uncertainの種はカウントしていない

env_feature_threshold_p = []
for j, env in enumerate(env_list):
    for i, md in enumerate(md_list):
        for threshold in [0]:
            Env = env+str(threshold)
            branch_count = \
                [
                    [0,0],
                    [0,0]
                ]
            for internal_node in tree.get_nonterminals()[1:]:
                sp = internal_node.name
                for child_node in internal_node.clades:
                    sp_child = child_node.name
                    if (spenv2exist(sp, Env, sp2env2exist) != 0.5):
                        exist    = (spenv2exist(sp, Env, sp2env2exist) == 1)
                        acquired = (spmd2Nrn(sp, md, sp2md2Nrn) < spmd2Nrn(sp_child, md, sp2md2Nrn))
                        branch_count[exist][acquired] += 1
            oddsratio, pvalue = stats.fisher_exact(branch_count, alternative='two-sided')
            env_feature_threshold_p.append([env, md, threshold, branch_count, oddsratio, pvalue])

# 環境進出(soil +/ rhizosphere -)　→　反応獲得　の傾向があるか検定
for i, md in enumerate(md_list):
    for threshold in [0]:
        env= "soil"
        Env = env+str(threshold)
        branch_count = \
            [
                [0,0],
                [0,0]
            ]
        for internal_node in tree.get_nonterminals()[1:]:
            sp = internal_node.name
            for child_node in internal_node.clades:
                sp_child = child_node.name
                if (spenv2exist(sp, Env, sp2env2exist) != 0.5):
                    exist    = (spenv2exist(sp, Env, sp2env2exist) == 1) * (spenv2exist(sp, "rhizosphere"+str(threshold), sp2env2exist) == 0)
                    acquired = (spmd2Nrn(sp, md, sp2md2Nrn) < spmd2Nrn(sp_child, md, sp2md2Nrn))
                    branch_count[exist][acquired] += 1
        oddsratio, pvalue = stats.fisher_exact(branch_count, alternative='two-sided')
        env_feature_threshold_p.append(["soil + rhizosphere -", md, threshold, branch_count, oddsratio, pvalue])

# 環境進出(soil +/ plant -)　→　反応獲得　の傾向があるか検定
for i, md in enumerate(md_list):
    for threshold in [0]:
        env= "soil"
        Env = env+str(threshold)
        branch_count = \
            [
                [0,0],
                [0,0]
            ]
        for internal_node in tree.get_nonterminals()[1:]:
            sp = internal_node.name
            for child_node in internal_node.clades:
                sp_child = child_node.name
                if (spenv2exist(sp, Env, sp2env2exist) != 0.5):
                    exist    = (spenv2exist(sp, Env, sp2env2exist) == 1) * (spenv2exist(sp, "plant"+str(threshold), sp2env2exist) == 0)
                    acquired = (spmd2Nrn(sp, md, sp2md2Nrn) < spmd2Nrn(sp_child, md, sp2md2Nrn))
                    branch_count[exist][acquired] += 1
        oddsratio, pvalue = stats.fisher_exact(branch_count, alternative='two-sided')
        env_feature_threshold_p.append(["soil + plant -", md, threshold, branch_count, oddsratio, pvalue])
        
# multiple test
df_env_to_feature_test = pd.DataFrame(env_feature_threshold_p, columns = ["Environment", "Feature", "Threshold", "table", "OddsRatio", "p"])
df_env_to_feature_test["q"] = multipletests(list(df_env_to_feature_test['p']), method = 'fdr_bh')[1]
df_env_to_feature_test["sig"] = ["*" if q < 0.05 else "" for q in df_env_to_feature_test["q"]]
df_env_to_feature_test["color"] = ["#FF0000" if (odds > 1) else "#0000FF" for odds in df_env_to_feature_test["OddsRatio"]]

df_env_to_feature_test.to_csv("tables/result_env2md.txt", index=False, sep = '\t')
df_env_to_feature_test

In [ ]:
# 全てのペアモジュールについて結果を可視化
Threshold_list = [0]
for j, env in enumerate(df_env_to_feature_test["Environment"]):

    df_env_to_feature_test_ext = df_env_to_feature_test[df_env_to_feature_test["Environment"]==env]
    df_env_to_feature_test_ext.reindex(index = feature_list)
    
    fig = plt.figure(figsize=(3,1.5))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    ax.bar(x=df_env_to_feature_test_ext["Feature"], height=-np.log10(df_env_to_feature_test_ext["q"]),width=0.8,color=df_env_to_feature_test_ext["color"], alpha = 0.5)
    #ax.set_xticklabels(md_list,rotation=90)
    ax.tick_params(axis='x', labelrotation= 90)
    ax.set_ylabel("-log$_{10}$ adj. p-value")
    ax.set_title(env)
    ax.set_ylim(0,15)
    ax.axhline(-np.log10(0.05))
    for x, sig in enumerate(df_env_to_feature_test_ext["sig"]):
        ax.text(x, 12.5, sig, horizontalalignment='center', fontsize = 15)
    fig.savefig("images/NK_M0155_test_env2feature_"+env+".pdf", bbox_inches='tight')
    plt.close()
    
    '''
    #nearly_zero= 10**-10
    fig = plt.figure(figsize=(3,1.5))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    ax.bar(x=df_env_to_feature_test_ext["Feature"], height=[ratio for ratio in df_env_to_feature_test_ext["OddsRatio"]],width=0.8,color="#005493")
    #ax.set_xticklabels(md_list,rotation=90)
    ax.tick_params(axis='x', labelrotation= 90)
    ax.set_ylabel("Odds Ratio")
    ax.set_title(env)
    #ax.set_ylim(0, 100)
    ax.axhline(1)
    for x, sig in enumerate(df_env_to_feature_test_ext["sig"]):
        ax.text(x, max(df_env_to_feature_test_ext["OddsRatio"]), sig, horizontalalignment='center', fontsize = 15)
    fig.savefig("images/NK_M0155_test_env2feature_"+env+"_odds.pdf", bbox_inches='tight')
    plt.close()
    '''
    

In [ ]:
# カテコールを中間産物とするペアについて結果を可視化

catechol_associated_modules = ["M00637", "M00551", "M00548",  "M00547", "M00569", "M00568"]

for j, env in enumerate(df_env_to_feature_test["Environment"]):
    
    df_env_to_feature_test_ext = df_env_to_feature_test[df_env_to_feature_test["Environment"]==env].reset_index()
    #df_env_to_feature_test_ext.reindex(index = feature_list)
    
    fig = plt.figure(figsize=(2,0.9))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])

    df_env_to_feature_test_ext["-logq"] = -np.log10(df_env_to_feature_test_ext["q"])
    df_env_to_feature_test_ext.index = df_env_to_feature_test_ext["Feature"]
    df_env_to_feature_test_ext_ext = df_env_to_feature_test_ext.reindex(index = catechol_associated_modules)
    ax.bar(df_env_to_feature_test_ext_ext["Feature"], df_env_to_feature_test_ext_ext["-logq"], color=df_env_to_feature_test_ext_ext["color"],alpha = 0.5)
    ax.axhline(-np.log10(0.05), alpha = 0.5, linewidth=0.75)
    #for x, sig in enumerate(df_env_to_feature_test_ext_ext["sig"]):
    #       ax.text(x, 3, sig, horizontalalignment='center', fontsize = 15)

    ax.tick_params(axis='x', labelrotation= 90)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    ax.set_title(env, y = 0.8, fontsize=10)
    ax.set_yticks([0,2, 4,6, 8,10])
    ax.set_ylabel("-log$_{10}$ adj. p")
    fig.savefig("images/NK_M0155_test_env2feature_"+env+".catechol.pdf", bbox_inches='tight')
    plt.close()

In [ ]:
# カテコールを中間産物とするペアについて結果を可視化 (M00547以外。進化学会スライド用)

catechol_associated_modules = ["M00637", "M00551", "M00548", "M00569", "M00568"]

for j, env in enumerate(df_env_to_feature_test["Environment"]):
    
    df_env_to_feature_test_ext = df_env_to_feature_test[df_env_to_feature_test["Environment"]==env].reset_index()
    #df_env_to_feature_test_ext.reindex(index = feature_list)
    
    fig = plt.figure(figsize=(2,0.9))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])

    df_env_to_feature_test_ext["-logq"] = -np.log10(df_env_to_feature_test_ext["q"])
    df_env_to_feature_test_ext.index = df_env_to_feature_test_ext["Feature"]
    df_env_to_feature_test_ext_ext = df_env_to_feature_test_ext.reindex(index = catechol_associated_modules)
    ax.bar(df_env_to_feature_test_ext_ext["Feature"], df_env_to_feature_test_ext_ext["-logq"], color=df_env_to_feature_test_ext_ext["color"],alpha = 0.5)
    ax.axhline(-np.log10(0.05), alpha = 0.5, linewidth=0.75)
    #for x, sig in enumerate(df_env_to_feature_test_ext_ext["sig"]):
    #       ax.text(x, 3, sig, horizontalalignment='center', fontsize = 15)

    ax.tick_params(axis='x', labelrotation= 90)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    ax.set_title(env, y = 0.8, fontsize=10)
    ax.set_yticks([0,2, 4,6, 8,10])
    ax.set_ylabel("-log$_{10}$ adj. p")
    fig.savefig("images/NK_M0155_test_env2feature_"+env+".catechol.without_M00547.pdf", bbox_inches='tight')
    plt.close()

In [ ]:
df_env_to_feature_test_ext

In [ ]:
# 直近の祖先が環境Xに生息している枝で、環境Yに進出する傾向があるか否か検定
env_feature_threshold_p = []
for envX, envY in [("soil", "rhizosphere"), ("soil", "sludge"), ("soil", "biosolids"), ("soil", "plant"), ("rhizosphere", "soil"), ("sludge", "soil"), ("biosolids", "soil"), ("plant", "soil")]:
        for threshold in [0]:
            EnvX = envX+str(threshold)
            EnvY = envY+str(threshold)
            branch_count = \
                [
                    [0,0],
                    [0,0]
                ]
            for internal_node in tree.get_nonterminals()[1:]:
                sp = internal_node.name
                for child_node in internal_node.clades:
                    sp_child = child_node.name
                    if (spenv2exist(sp, EnvY, sp2env2exist) == 0):
                        if (spenv2exist(sp, EnvX, sp2env2exist) != 0.5):
                            exist    = (spenv2exist(sp, EnvX, sp2env2exist) == 1)
                            #spread2Y = (spfeature2Nrn(sp, EnvY, sp2env2exist) < spfeature2Nrn(sp_child, EnvY, sp2env2exist) )
                            spread2Y = (spenv2exist(sp, EnvY, sp2env2exist) < spenv2exist(sp_child, EnvY, sp2env2exist) )
                            
                            branch_count[exist][spread2Y] += 1
            print(envX, envY, branch_count)
            oddsratio, pvalue = stats.fisher_exact(branch_count)
            env_feature_threshold_p.append([envX + " > " + envY, envX, envY, threshold, oddsratio, pvalue])

# multiple test
df_envX_to_envY_test = pd.DataFrame(env_feature_threshold_p, columns = ["name", "EnvironmentX", "EnvironmentY", "Threshold", "OddsRatio",  "p"])
df_envX_to_envY_test["q"] = multipletests(list(df_envX_to_envY_test['p']), method = 'fdr_bh')[1]
df_envX_to_envY_test["sig"] = ["*" if q < 0.01 else "" for q in df_envX_to_envY_test["q"]]
df_envX_to_envY_test["color"] = ["#FF0000" if (odds > 1) else "#0000FF" for odds in df_envX_to_envY_test["OddsRatio"]]



# visualization

Threshold_list = [0]

fig = plt.figure(figsize=(2.7,1.5))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
ax.bar(x=df_envX_to_envY_test["name"], height=-np.log10(df_envX_to_envY_test["q"]),width=0.5,color=df_envX_to_envY_test["color"], alpha = 0.5)
ax.tick_params(axis='x', labelrotation=90)
ax.set_ylabel("-log$_{10}$ adj. p-value")
ax.set_ylim(0,10)
ax.axhline(-np.log10(0.05), alpha = 0.5)
fig.savefig("images/NK_M0155_test_env2env.pdf", bbox_inches='tight')
#plt.show()
plt.close()

df_envX_to_envY_test

In [ ]:
# 環境進出(soil +/ rhizosphere -)　→　反応獲得　の傾向があるか検定
# 直近の祖先がその環境に生息している枝で、反応が獲得される傾向があるか否か検定
# 環境生息の祖先状態が1の種を生息しているとし、環境生息の祖先状態が0の種を生息していないとする、uncertainの種はカウントしていない

env_feature_threshold_p = []
for i, md in enumerate(md_list):
    for threshold in [0]:
        env= "soil"
        Env = env+str(threshold)
        branch_count = \
            [
                [0,0],
                [0,0]
            ]
        for internal_node in tree.get_nonterminals()[1:]:
            sp = internal_node.name
            for child_node in internal_node.clades:
                sp_child = child_node.name
                if (spenv2exist(sp, Env, sp2env2exist) != 0.5):
                    exist    = (spenv2exist(sp, env, sp2env2exist) == 1) * (spenv2exist(sp, "rhizosphere"+str(threshold), sp2env2exist) == 0)
                    acquired = (spmd2Nrn(sp, md, sp2md2Nrn) < spmd2Nrn(sp_child,md, sp2md2Nrn) )
                    branch_count[exist][acquired] += 1
        oddsratio, pvalue = stats.fisher_exact(branch_count, alternative='two-sided')
        print(branch_count)
        env_feature_threshold_p.append([env, md, threshold, pvalue])

# multiple test
df_env_to_feature_test = pd.DataFrame(env_feature_threshold_p, columns = ["Environment", "Feature", "Threshold", "p"])
df_env_to_feature_test["q"] = multipletests(list(df_env_to_feature_test['p']), method = 'fdr_bh')[1]
df_env_to_feature_test["sig"] = ["*" if q < 0.01 else "" for q in df_env_to_feature_test["q"]]

# visualization
Threshold_list = [0]
for j, env in enumerate(["soil"]):

    df_env_to_feature_test_ext = df_env_to_feature_test[df_env_to_feature_test["Environment"]==env]
    df_env_to_feature_test_ext.reindex(index = feature_list)
    
    fig = plt.figure(figsize=(3,1.5))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    ax.bar(x=df_env_to_feature_test_ext["Feature"], height=-np.log10(df_env_to_feature_test_ext["q"]),width=0.8,color="#9A9AFF")
    ax.set_xticklabels(md_list,rotation=90)
    ax.set_ylabel("-log$_{10}$ adj. p-value")
    ax.set_title("soil + / rhizosphere -")
    ax.set_ylim(0,15)
    ax.axhline(-np.log10(0.01))
    for x, sig in enumerate(df_env_to_feature_test_ext["sig"]):
        ax.text(x, 12.5, sig, horizontalalignment='center', fontsize = 15)
    fig.savefig("images/NK_M0155_test_env2feature_soil+rhizosphere-.pdf", bbox_inches='tight')
    plt.close()

In [ ]:
branch_count

In [ ]:
stats.fisher_exact([[10,0],[0,10]], alternative='greater')

#### 欠失進化のパターンについて、特定の機能モジュールに注目

In [ ]:
feature_list = [
    "M00022", "M00023",
    "M00018", "M00570",
]

env_list = list(env_set)

#### モジュール内反応数と各環境のhabitatbility scoreとの相関をとる

In [ ]:
# Calculation of the correlation coefficient

list_module_env_coeff = []

for module in list(set(feature_list)):
    df_Nreaction_ext = df_Nreaction[df_Nreaction["module"]==module]
    
    for env in env_list:
        
        df_sp_env_score_ext = df_sp_env_score[df_sp_env_score["environment"]==env]

        df_sp_md_env_score_ext = pd.merge(df_Nreaction_ext, df_sp_env_score_ext, on = 'species', how = 'right')
        df_sp_md_env_score_ext['module'] = module
        df_sp_md_env_score_ext = df_sp_md_env_score_ext.fillna(0) # convert Nreaction = n.a. -> 0
        df_sp_md_env_score_ext

        if (np.var(df_sp_md_env_score_ext['Nreaction']) == 0 and np.var(df_sp_md_env_score_ext['score'])==0):
            coeff = 0
        else:
            coeff = df_sp_md_env_score_ext['Nreaction'].corr(df_sp_md_env_score_ext['score'])

        list_module_env_coeff.append([module, env, coeff])
        
df_module_env_coeff = pd.DataFrame(list_module_env_coeff, columns = ["Feature", "Environment", "Coefficient"])
df_module_env_coeff

In [ ]:
# Visualization of the result
for feature in feature_list:
    feature_env_coeff_dataframe_ext = df_module_env_coeff[df_module_env_coeff["Feature"]==feature]
    feature_env_coeff_dataframe_ext = feature_env_coeff_dataframe_ext.sort_values('Coefficient')

    fig = plt.figure(figsize=(15,2))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    sns.barplot(x = 'Environment', y = 'Coefficient', data = feature_env_coeff_dataframe_ext, palette="coolwarm")
    ax.set_title(feature)
    ax.set_xlabel("Environment")
    ax.set_ylim(-0.5,0.5)
    plt.xticks(rotation=90)
    plt.savefig("images/NK_M0155_func_habit_coeff_"+feature+".pdf",bbox_inches='tight')
    plt.close()

In [ ]:
# Analyze the result

env_list = list(set(df_module_env_coeff[df_module_env_coeff["Coefficient"] < -0.1]["Environment"]))

df_module_env_coeff[df_module_env_coeff["Coefficient"] < -0.10]

In [ ]:
# featureとenvの相関を、scatter plot
for feature in feature_list:
    df[feature] = [spmd2Nrn(idx.split(":")[1].split("_")[0], feature, sp2md2Nrn) for idx in df.index]
df.to_csv("tables/table3.csv")

md_list = list(set(feature_list))

fig = plt.figure(figsize=(3,3))
for i, md in enumerate(md_list):
        for j, env in enumerate(env_list):
            ax = fig.add_axes([0.1+i*0.9,0.1+j*0.9,0.8,0.8])
            #ax = sns.violinplot( x=df["md:M00569"], y=df["soil"], linewidth=0)
            ax = sns.boxplot(x=df[md], y=df[env], color='white', linewidth=2)
            #ax = sns.swarmplot(x=df[md], y=df[env],alpha=1,size=1,color='black')
            ax.set_ylim(0,max(df[env]))
            if(i>0):
                ax.set_ylabel("")
                ax.tick_params(labelleft=False, left=False)
            if (j>0):
                ax.set_xlabel("")
                ax.tick_params(labelbottom=False, bottom=False)
plt.savefig("images/NK_M0155_Nrn_env_scactter_loss.pdf", bbox_inches='tight')
plt.close()

#### 環境進出　→　反応欠失　の傾向があるか検定

In [ ]:
# 環境進出　→　反応獲得　の傾向があるか検定
# 直近の祖先がその環境に生息している枝で、反応が獲得される傾向があるか否か検定
# 環境生息の祖先状態が1の種を生息しているとし、環境生息の祖先状態が0の種を生息していないとする、uncertainの種はカウントしていない

env_feature_threshold_p = []
for j, env in enumerate(env_list):
    for i, md in enumerate(md_list):
        for threshold in [0]:
            Env = env+str(threshold)
            branch_count = \
                [
                    [0,0],
                    [0,0]
                ]
            for internal_node in tree.get_nonterminals()[1:]:
                sp = internal_node.name
                for child_node in internal_node.clades:
                    sp_child = child_node.name
                    if (spenv2exist(sp, Env, sp2env2exist) != 0.5):
                        if (spmd2Nrn(sp, md, sp2md2Nrn) > 0):
                            exist    = (spenv2exist(sp, Env, sp2env2exist) == 1)
                            lost = (spmd2Nrn(sp, md, sp2md2Nrn) > spmd2Nrn(sp_child, md, sp2md2Nrn))
                            branch_count[exist][lost] += 1
            oddsratio, pvalue = stats.fisher_exact(branch_count, alternative='two-sided')
            env_feature_threshold_p.append([env +" +", md, threshold, branch_count, oddsratio, pvalue])

# multiple test
df_env_to_feature_test = pd.DataFrame(env_feature_threshold_p, columns = ["Environment", "Feature", "Threshold", "table", "OddsRatio", "p"])
df_env_to_feature_test["q"] = multipletests(list(df_env_to_feature_test['p']), method = 'fdr_bh')[1]
df_env_to_feature_test["sig"] = ["*" if q < 0.05 else "" for q in df_env_to_feature_test["q"]]
df_env_to_feature_test["color"] = ["#FF0000" if (odds > 1) else "#0000FF" for odds in df_env_to_feature_test["OddsRatio"]]

df_env_to_feature_test.to_csv("tables/result_env2md_loss.txt", index=False, sep = '\t')
df_env_to_feature_test

In [ ]:
# visualization
Threshold_list = [0]
for j, env in enumerate(df_env_to_feature_test["Environment"]):

    df_env_to_feature_test_ext = df_env_to_feature_test[df_env_to_feature_test["Environment"]==env]
    df_env_to_feature_test_ext.reindex(index = feature_list)
    
    fig = plt.figure(figsize=(0.75,1.5))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    ax.bar(x=df_env_to_feature_test_ext["Feature"], height=-np.log10(df_env_to_feature_test_ext["q"]),width=0.8,color=df_env_to_feature_test_ext["color"], alpha = 0.5)
    #ax.set_xticklabels(md_list,rotation=90)
    ax.tick_params(axis='x', labelrotation= 90)
    ax.set_ylabel("-log$_{10}$ adj. p-value")
    ax.set_title(env)
    ax.set_ylim(0,15)
    ax.axhline(-np.log10(0.05))
    for x, sig in enumerate(df_env_to_feature_test_ext["sig"]):
        ax.text(x, 12.5, sig, horizontalalignment='center', fontsize = 15)
    fig.savefig("images/NK_M0155_test_loss_env2feature_"+env+".pdf", bbox_inches='tight')
    plt.close()

In [ ]:
'''

# 直近の祖先が環境Xに生息している枝で、環境Yに進出する傾向があるか否か検定
env_feature_threshold_p = []
for envX, envY in [("insect_gut", "bovine_gut"), ("insect_gut", "chicken_gut"), ("insect_gut", "human_oral"), ("bovine_gut", "insect_gut"), ("chicken_gut", "insect_gut"), ("human_oral", "insect_gut")]:
        for threshold in [0]:
            EnvX = envX+str(threshold)
            EnvY = envY+str(threshold)
            branch_count = \
                [
                    [0,0],
                    [0,0]
                ]
            for internal_node in tree.get_nonterminals()[1:]:
                sp = internal_node.name
                for child_node in internal_node.clades:
                    sp_child = child_node.name
                    if (spenv2exist(sp, EnvY, sp2env2exist) == 0):
                        if (spenv2exist(sp, EnvX, sp2env2exist) != 0.5):
                            exist    = (spenv2exist(sp, EnvX, sp2env2exist) == 1)
                            #spread2Y = (spfeature2Nrn(sp, EnvY, sp2env2exist) < spfeature2Nrn(sp_child, EnvY, sp2env2exist) )
                            spread2Y = (spenv2exist(sp, EnvY, sp2env2exist) < spenv2exist(sp_child, EnvY, sp2env2exist) )
                            
                            branch_count[exist][spread2Y] += 1
            print(envX, envY, branch_count)
            oddsratio, pvalue = stats.fisher_exact(branch_count)
            env_feature_threshold_p.append([envX + " > " + envY, envX, envY, threshold, oddsratio, pvalue])

# multiple test
df_envX_to_envY_test = pd.DataFrame(env_feature_threshold_p, columns = ["name", "EnvironmentX", "EnvironmentY", "Threshold", "OddsRatio",  "p"])
df_envX_to_envY_test["q"] = multipletests(list(df_envX_to_envY_test['p']), method = 'fdr_bh')[1]
df_envX_to_envY_test["sig"] = ["*" if q < 0.05 else "" for q in df_envX_to_envY_test["q"]]
df_envX_to_envY_test["color"] = ["#FF0000" if (odds > 1) else "#0000FF" for odds in df_envX_to_envY_test["OddsRatio"]]



# visualization

Threshold_list = [0]

fig = plt.figure(figsize=(1.5,1.5))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
ax.bar(x=df_envX_to_envY_test["name"], height=-np.log10(df_envX_to_envY_test["q"]),width=0.5,color=df_envX_to_envY_test["color"], alpha = 0.5)
ax.tick_params(axis='x', labelrotation=90)
ax.set_ylabel("-log$_{10}$ adj. p-value")
ax.set_ylim(0,10)
ax.axhline(-np.log10(0.05))
for x, sig in enumerate(df_envX_to_envY_test["sig"]):
    ax.text(x, 8, sig, horizontalalignment='center', fontsize = 15)

fig.savefig("images/NK_M0155_test_loss_env2env.pdf", bbox_inches='tight')
##plt.show()
#plt.close()

df_envX_to_envY_test
'''